# Lecture 10: Pandas Basics

Numpy是数据分析的基础库，pandas则是后续数据分析的首选库。它能使数据清洗和分析工作变得更快更简单。pandas也会经常和其它工具包一起使用，如数值计算包Numpy和SciPy，统计分析包statsmodels和scikit-learn，以及数据可视化库matplotlib。

pandas是基于NumPy数组构建的，特别是基于数组的函数和不使用for循环的数据处理。

虽然pandas采用了大量的NumPy编码风格，但二者最大的不同是pandas是专门为处理表格和混杂数据设计的。而NumPy更适合处理统一的数值数组数据。

自从2010年pandas开源以来，pandas逐渐成长为一个非常大的库，应用于许多真实案例。开发者社区已经有了800个独立的贡献者，他们在解决日常数据问题的同时为这个项目提供贡献。

In [ ]:
import pandas as pd

只要你在代码中看到pd.，就得想到这是pandas。因为Series和DataFrame用的次数非常多，所以将其引入本地命名空间中会更方便：

In [ ]:
from pandas import Series, DataFrame

In [ ]:
import numpy as np
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

## 1 pandas的数据结构介绍
要使用pandas，你首先就得熟悉它的两个主要数据结构：Series和DataFrame。

### Series

Series是一种类似于一维数组的对象，它由一组数据（各种NumPy数据类型）以及一组与之相关的数据标签（即索引）组成。仅由一组数据即可产生最简单的Series：

In [ ]:
obj = pd.Series([4, 7, -5, 3])
obj

Series的字符串表现形式为：索引在左边，值在右边。由于我们没有为数据指定索引，于是会自动创建一个0到N-1（N为数据的长度）的整数型索引。你可以通过Series 的values和index属性获取其数组表示形式和索引对象：

In [ ]:
obj.values

In [ ]:
obj.index

通常，我们希望所创建的Series带有一个可以对各个数据点进行标记的索引：

In [ ]:
obj2 = pd.Series([4, 7, -5, 3], index=["d", "b", "a", "c"])
obj2

In [ ]:
obj2.index

与普通NumPy数组相比，你可以通过索引的方式选取Series中的单个或一组值：

In [ ]:
obj2["a"]

In [ ]:
obj2["d"] = 6
obj2[["c", "a", "d"]]

['c', 'a', 'd']是索引列表，即使它包含的是字符串而不是整数。

使用NumPy函数或类似NumPy的运算（如根据布尔型数组进行过滤、标量乘法、应用数学函数等）都会保留索引值的链接：

In [ ]:
obj2[obj2 > 0]

In [ ]:
obj2 * 2

In [ ]:
np.exp(obj2)

还可以将Series看成是一个定长的有序字典，因为它是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中：

In [ ]:
"b" in obj2

In [ ]:
"e" in obj2

如果数据被存放在一个Python字典中，也可以直接通过这个字典来创建Series：

In [ ]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}
obj3 = pd.Series(sdata)
obj3

In [ ]:
obj3.to_dict()

如果只传入一个字典，则Series中的索引就是原字典的键（有序排列）。你也可以传入字典的键至index，以改变顺序：

In [ ]:
states = ["California", "Ohio", "Oregon", "Texas"]
obj4 = pd.Series(sdata, index=states)
obj4

pandas的isnull和notnull函数可用于检测缺失数据：

In [ ]:
pd.isna(obj4)

In [ ]:
pd.notna(obj4)

Series也有类似的实例方法：

In [ ]:
obj4.isna()

对于许多应用而言，Series最重要的一个功能是，它会根据运算的索引标签自动对齐数据：

In [ ]:
obj3

In [ ]:
obj4

In [ ]:
obj3 + obj4

Series对象本身及其索引都有一个name属性，该属性跟pandas其他的关键功能关系非常密切：

In [ ]:
obj4.name = "population"
obj4.index.name = "state"
obj4

Series的索引可以通过赋值的方式就地修改：

In [ ]:
obj

In [ ]:
obj.index = ["Bob", "Steve", "Jeff", "Ryan"]
obj

### DataFrame
DataFrame是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值、字符串、布尔值等）。DataFrame既有行索引也有列索引，它可以被看做由Series组成的字典（共用同一个索引）。DataFrame中的数据是以一个或多个二维块存放的（而不是列表、字典或别的一维数据结构）。

虽然DataFrame是以二维结构保存数据的，但你仍然可以轻松地将其表示为更高维度的数据（层次化索引的表格型结构，这是pandas中许多高级数据处理功能的关键要素，

建DataFrame的办法有很多，最常用的一种是直接传入一个由等长列表或NumPy数组组成的字典：

In [ ]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}

frame = pd.DataFrame(data)
frame

对于特别大的DataFrame，head方法会选取前五行：

In [ ]:
frame.head()

In [ ]:
# 选最后五行
frame.tail()

如果指定了列序列，则DataFrame的列就会按照指定顺序进行排列：

In [ ]:
pd.DataFrame(data, columns=["year", "state", "pop"])

如果传入的列在数据中找不到，就会在结果中产生缺失值：

In [ ]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four', 'five', 'six'])
frame2

通过类似字典标记的方式或属性的方式，可以将DataFrame的列获取为一个Series：

In [ ]:
frame2["state"]

In [ ]:
frame2.year

行也可以通过位置或名称的方式进行获取，比如用loc和iloc属性（稍后将对此进行详细讲解）：

In [ ]:
frame2.loc['three'] # row name

In [ ]:
frame2.iloc[2] # row index

列可以通过赋值的方式进行修改。例如，我们可以给那个空的"debt"列赋上一个标量值或一组值：

In [ ]:
frame2["debt"] = 16.5
frame2

In [ ]:
frame2["debt"] = np.arange(6.)
frame2

将列表或数组赋值给某个列时，其长度必须跟DataFrame的长度相匹配。如果赋值的是一个Series，就会精确匹配DataFrame的索引，所有的空位都将被填上缺失值：

In [ ]:
val = pd.Series([-1.2, -1.5, -1.7], index=["two", "four", "five"])
frame2["debt"] = val
frame2

为不存在的列赋值会创建出一个新列。关键字del用于删除列。

作为del的例子，我先添加一个新的布尔值的列，state是否为'Ohio'：

In [ ]:
frame2["eastern"] = frame2["state"] == "Ohio"
frame2

In [ ]:
del frame2["eastern"]
frame2.columns

另一种常见的数据形式是嵌套字典：

In [ ]:
populations = {"Ohio": {2000: 1.5, 2001: 1.7, 2002: 3.6},
               "Nevada": {2001: 2.4, 2002: 2.9}}

如果嵌套字典传给DataFrame，pandas就会被解释为：外层字典的键作为列，内层键则作为行索引：

In [ ]:
frame3 = pd.DataFrame(populations)
frame3

你也可以使用类似NumPy数组的方法，对DataFrame进行转置（交换行和列）：

In [ ]:
frame3.T

内层字典的键会被合并、排序以形成最终的索引。如果明确指定了索引，则不会这样：

In [ ]:
pd.DataFrame(populations, index=[2001, 2002, 2003])

如果DataFrame各列的数据类型不同，则值数组的dtype就会选用能兼容所有列的数据类型：

In [ ]:
frame2.values

## 索引对象
pandas的索引对象负责管理轴标签和其他元数据（比如轴名称等）。构建Series或DataFrame时，所用到的任何数组或其他序列的标签都会被转换成一个Index：

In [ ]:
obj = pd.Series(np.arange(3), index=["a", "b", "c"])
idx = obj.index
idx

Index对象是不可变的，因此用户不能对其进行修改。
不可变可以使Index对象在多个数据结构之间安全共享。

In [ ]:
idx[1] = 'd'  # TypeError

## 2 基本功能
本节中，我们将介绍操作Series和DataFrame中数据操作的基本功能。后续将更加深入地挖掘pandas在数据分析方面的功能。但是，本节内容并不是pandas库的详尽文档，关注的是最重要的功能，那些不大常用的内容（也就是更深奥的内容）就交给你自己去探索吧。

### 重新索引
pandas对象的一个重要方法是reindex，其作用是创建一个新对象，它的数据符合新的索引。看下面的例子：

In [ ]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=["d", "b", "a", "c"])
obj

用该Series的reindex将会根据新索引进行重排。如果某个索引值当前不存在，就引入缺失值：

In [ ]:
obj2 = obj.reindex(["a", "b", "c", "d", "e"])
obj2

对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理。method选项即可达到此目的，例如，使用ffill可以实现前向值填充：

In [ ]:
obj3 = pd.Series(["blue", "purple", "yellow"], index=[0, 2, 4])
obj3

In [ ]:
obj3.reindex(np.arange(6), method="ffill")

借助DataFrame，reindex可以修改（行）索引和列。只传递一个序列时，会重新索引结果的行：

In [ ]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=["a", "c", "d"],
                     columns=["Ohio", "Texas", "California"])
frame

In [ ]:
frame2 = frame.reindex(index=["a", "b", "c", "d"])
frame2

In [ ]:
states = ["Texas", "Utah", "California"]
frame.reindex(columns=states)

### 丢弃指定轴上的项
丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。由于需要执行一些数据整理和集合逻辑，所以drop方法返回的是一个在指定轴上删除了指定值的新对象：

In [ ]:
obj = pd.Series(np.arange(5.), index=["a", "b", "c", "d", "e"])
obj

In [ ]:
new_obj = obj.drop("c")
new_obj

In [ ]:
obj.drop(["d", "c"])

对于DataFrame，可以删除任意轴上的索引值。为了演示，我们先新建一个DataFrame例子：

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data

用标签序列调用drop会从行标签（axis 0）删除值：

In [ ]:
data.drop(index=["Colorado", "Ohio"])

In [ ]:
data.drop(columns=["two"])

也可以通过传递axis=1或axis='columns'可以删除列的值：

In [ ]:
data.drop("two", axis=1)

In [ ]:
data.drop(["two", "four"], axis="columns")

许多函数，如drop，会修改Series或DataFrame的大小或形状，可以就地修改对象，不会返回新的对象：\
(注意：请小心使用inplace，它会销毁所有被删除的数据。)

In [ ]:
obj.drop('c', inplace=True)
obj

### 索引、选取和过滤
Series索引（obj[...]）的工作方式类似于NumPy数组的索引，只不过Series的索引值不只是整数。下面是几个例子：

In [ ]:
obj = pd.Series(np.arange(4.), index=["a", "b", "c", "d"])
obj

In [ ]:
obj["b"]

In [ ]:
obj[1]

In [ ]:
obj[2:4]

In [ ]:
obj[["b", "a", "d"]]

In [ ]:
obj[[1, 3]]

In [ ]:
obj[obj < 2]

### 用loc和iloc进行选取
对于DataFrame的行的标签索引，pandas的引入了特殊的标签运算符loc和iloc。它们可以让你用类似NumPy的标记，使用轴标签（loc）或整数索引（iloc），从DataFrame选择行和列的子集。

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data

In [ ]:
data.loc["Colorado"]

In [ ]:
data.loc[["Colorado", "New York"]]

In [ ]:
data.loc["Colorado", ["two", "three"]]

用iloc和整数进行选取：

In [ ]:
data.iloc[2]

In [ ]:
data.iloc[[2, 1]]

In [ ]:
data.iloc[2, [3, 0, 1]]

In [ ]:
data.iloc[[1, 2], [3, 0, 1]]

这两个索引函数也适用于一个标签或多个标签的切片：

In [ ]:
data.loc[:"Utah", "two"]

In [ ]:
data.iloc[:, :3][data.three > 5]

所以，在pandas中，有多个方法可以选取和重新组合数据。对于DataFrame，下表进行了总结。后面会看到，还有更多的方法进行层级化索引。

![pandas_index](figures/pandas_index.jpg)

### 算术运算和数据对齐
pandas最重要的一个功能是，它可以对不同索引的对象进行算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。对于有数据库经验的用户，这就像在索引标签上进行自动外连接。看一个简单的例子：

In [ ]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=["a", "c", "d", "e"])
s1

In [ ]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=["a", "c", "e", "f", "g"])
s2

In [ ]:
s1 + s2

In [ ]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list("bcd"),
                   index=["Ohio", "Texas", "Colorado"])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list("bde"),
                   index=["Utah", "Ohio", "Texas", "Oregon"])

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1 + df2

如果DataFrame对象相加，没有共用的列或行标签，结果都会是空：

In [ ]:
df1 = pd.DataFrame({"A": [1, 2]})
df2 = pd.DataFrame({"B": [3, 4]})
df1

In [ ]:
df2

In [ ]:
df1 + df2

### 在算术方法中填充值
在对不同索引的对象进行算术运算时，你可能希望当一个对象中某个轴标签在另一个对象中找不到时填充一个特殊值（比如0）：

In [ ]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)), columns=list("abcd"))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)), columns=list("abcde"))
df2.loc[1, "b"] = np.nan

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1 + df2

使用df1的add方法，传入df2以及一个fill_value参数:\
相当于把df1中在df2不存在的位置都赋值为0后再与df2相加。

In [ ]:
df1.add(df2, fill_value=0)

In [ ]:
df1.add(df2)

与此类似，在对Series或DataFrame重新索引时，也可以指定一个填充值：

In [ ]:
df1.reindex(columns=df2.columns, fill_value=0)

### DataFrame和Series之间的运算
跟不同维度的NumPy数组一样，DataFrame和Series之间算术运算也是有明确规定的。先来看一个具有启发性的例子，计算一个二维数组与其某行之间的差：

In [ ]:
arr = np.arange(12.).reshape((3, 4))
arr

In [ ]:
arr[0]

In [ ]:
arr - arr[0]

当我们从arr减去arr[0]，每一行都会执行这个操作。这就叫做广播（broadcasting）。\
DataFrame和Series之间的运算差不多也是如此：

In [ ]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list("bde"),
                     index=["Utah", "Ohio", "Texas", "Oregon"])
frame

In [ ]:
series = frame.iloc[0]
series

默认情况下，DataFrame和Series之间的算术运算会将Series的索引匹配到DataFrame的列，然后沿着行一直向下广播：

In [ ]:
frame - series

如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集：

In [ ]:
series2 = pd.Series(np.arange(3), index=["b", "e", "f"])
series2

In [ ]:
frame + series2

如果你希望匹配行且在列上广播，则必须使用算术运算方法。例如：

In [ ]:
frame

In [ ]:
series3 = frame["d"]
series3

In [ ]:
frame.sub(series3, axis="index")

传入的轴号就是希望匹配的轴。在本例中，我们的目的是匹配DataFrame的行索引（axis='index' or axis=0）并进行广播。

### 函数应用和映射
NumPy的ufuncs（元素级数组方法）也可用于操作pandas对象：

In [ ]:
frame = pd.DataFrame(np.random.standard_normal((4, 3)),
                     columns=list("bde"),
                     index=["Utah", "Ohio", "Texas", "Oregon"])
frame

In [ ]:
np.abs(frame)

另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上。DataFrame的apply方法即可实现此功能：

In [ ]:
def f1(x):
    return x.max() - x.min()

frame.apply(f1)

这里的函数f，计算了一个Series的最大值和最小值的差，在frame的每列都执行了一次。结果是一个Series，使用frame的列作为索引。

如果传递axis='columns'到apply，这个函数会在每行执行：

In [ ]:
frame.apply(f1, axis="columns")

许多最为常见的数组统计功能都被实现成DataFrame的方法（如sum和mean），因此无需使用apply方法。

传递到apply的函数不是必须返回一个标量，还可以返回由多个值组成的Series：

In [ ]:
def f2(x):
    return pd.Series([x.min(), x.max()], index=["min", "max"])
frame.apply(f2)

元素级的Python函数也是可以用的。假如你想得到frame中各个浮点值的格式化字符串，使用applymap即可：

In [ ]:
def my_format(x):
    return f"{x:.2f}"

frame.applymap(my_format)

之所以叫做applymap，是因为Series有一个用于应用元素级函数的map方法：

In [ ]:
frame["e"].map(my_format)

### 排序和排名
根据条件对数据集排序（sorting）也是一种重要的内置运算。要对行或列索引进行排序（按字典顺序），可使用sort_index方法，它将返回一个已排序的新对象：

In [ ]:
obj = pd.Series(np.arange(4), index=["d", "a", "b", "c"])
obj

In [ ]:
obj.sort_index()

对于DataFrame，则可以根据任意一个轴上的索引进行排序：

In [ ]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=["three", "one"],
                     columns=["d", "a", "b", "c"])
frame

In [ ]:
frame.sort_index(axis="columns")

数据默认是按升序排序的，但也可以降序排序：

In [ ]:
frame.sort_index(axis="columns", ascending=False)

若要按值对Series进行排序，可使用其sort_values方法：

In [ ]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

在排序时，任何缺失值默认都会被放到Series的末尾：

In [ ]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

当排序一个DataFrame时，你可能希望根据一个或多个列中的值进行排序。将一个或多个列的名字传递给sort_values的by选项即可达到该目的：

In [ ]:
frame = pd.DataFrame({"b": [4, 7, -3, 2], "a": [0, 1, 0, 1]})
frame.sort_values(by="b")

要根据多个列进行排序，传入名称的列表即可：

In [ ]:
frame.sort_values(["a", "b"])

排名会从1开始一直到数组中有效数据的数量。接下来介绍Series和DataFrame的rank方法。默认情况下，rank是通过“为各组分配一个平均排名”的方式破坏平级关系的：

In [ ]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

也可以根据值在原数据中出现的顺序给出排名：

In [ ]:
obj.rank(method="first")

这里，条目0和2没有使用平均排名6.5，它们被设成了6和7，因为数据中标签0位于标签2的前面。

你也可以按降序进行排名：

In [ ]:
obj.rank(ascending=False)

In [ ]:
frame = pd.DataFrame({"b": [4.3, 7, -3, 2], "a": [0, 1, 0, 1],
                      "c": [-2, 5, 8, -2.5]})
frame

In [ ]:
frame.rank(axis="columns")

### 带有重复标签的轴索引
直到目前为止，我所介绍的所有范例都有着唯一的轴标签（索引值）。虽然许多pandas函数（如reindex）都要求标签唯一，但这并不是强制性的。我们来看看下面这个简单的带有重复索引值的Series：

In [ ]:
obj = pd.Series(np.arange(5), index=["a", "a", "b", "b", "c"])
obj

In [ ]:
obj.index.is_unique

对于带有重复值的索引，数据选取的行为将会有些不同。如果某个索引对应多个值，则返回一个Series；而对应单个值的，则返回一个标量值：

In [ ]:
obj["a"]

In [ ]:
obj["c"]

这样会使代码变复杂，因为索引的输出类型会根据标签是否有重复发生变化。

对DataFrame的行进行索引时也是如此：

In [ ]:
df = pd.DataFrame(np.random.standard_normal((5, 3)),
                  index=["a", "a", "b", "b", "c"])
df

In [ ]:
df.loc["b"]

In [ ]:
df.loc["c"]

## 汇总和计算描述统计
pandas对象拥有一组常用的数学和统计方法。它们大部分都属于约简和汇总统计，用于从Series中提取单个值（如sum或mean）或从DataFrame的行或列中提取一个Series。跟对应的NumPy数组方法相比，它们都是基于没有缺失数据的假设而构建的。看一个简单的DataFrame：

In [ ]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=["a", "b", "c", "d"],
                  columns=["one", "two"])
df

调用DataFrame的sum方法将会返回一个含有列的和的Series：

In [ ]:
df.sum()

传入axis='columns'或axis=1将会按行进行求和运算：

In [ ]:
df.sum(axis="columns")

NA值会自动被排除，除非整个切片（这里指的是行或列）都是NA。通过skipna选项可以禁用该功能：

In [ ]:
df.sum(axis="index", skipna=False)

In [ ]:
df.sum(axis="columns", skipna=False)

In [ ]:
df.mean(axis="columns")

有些方法（如idxmin和idxmax）返回的是间接统计（比如达到最小值或最大值的索引）：

In [ ]:
df.idxmax()

另一些方法则是累计型的：

In [ ]:
df.cumsum()

还有一种方法，它既不是约简型也不是累计型。describe就是一个例子，它用于一次性产生多个汇总统计：

In [ ]:
df.describe()

对于非数值型数据，describe会产生另外一种汇总统计：

In [ ]:
obj = pd.Series(["a", "a", "b", "c"] * 4)
obj.describe()

![pandas_describes](figures/pandas_describes.jpg)

## This is the end! :)